In [1]:
import pandas as pd
import numpy as np
from scipy.special import expit
from functools import reduce
from online_logistic_regression import OnlineLogisticRegression


In [2]:
from environment import ContextualEnvironment
from policies import KLUCBSegmentPolicy, RandomPolicy, ExploreThenCommitSegmentPolicy, EpsilonGreedySegmentPolicy, TSSegmentPolicy, LinearTSPolicy
import argparse
import json
import logging
import numpy as np
import pandas as pd
import time

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--users_path", type = str, default = "data/user_features.csv", required = False,
                    help = "Path to user features file")
parser.add_argument("--playlists_path", type = str, default = "data/playlist_features.csv", required = False,
                    help = "Path to playlist features file")
parser.add_argument("--output_path", type = str, default = "results.json", required = False,
                    help = "Path to json file to save regret values")
parser.add_argument("--policies", type = str, default = "random,ts-seg-naive", required = False,
                    help = "Bandit algorithms to evaluate, separated by commas")
parser.add_argument("--n_recos", type = int, default = 12, required = False,
                    help = "Number of slots L in the carousel i.e. number of recommendations to provide")
parser.add_argument("--l_init", type = int, default = 3, required = False,
                    help = "Number of slots L_init initially visible in the carousel")
parser.add_argument("--n_users_per_round", type = int, default = 20000, required = False,
                    help = "Number of users randomly selected (with replacement) per round")
parser.add_argument("--n_rounds", type = int, default = 100, required = False,
                    help = "Number of simulated rounds")
parser.add_argument("--print_every", type = int, default = 10, required = False,
                    help = "Print cumulative regrets every 'print_every' round")

args = parser.parse_args(args = [])

In [4]:
playlists_df = pd.read_csv('data/playlist_features.csv')

users_df = pd.read_csv('data/user_features_small.csv')

n_users = len(users_df)
n_playlists = len(playlists_df)

In [5]:
user_features = np.array(users_df.drop(["segment"], axis = 1))
user_features = np.concatenate([user_features, np.ones((n_users,1))], axis = 1)
playlist_features = np.array(playlists_df)

In [6]:
user_segment = np.array(users_df.segment)

In [156]:
user_segment

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [8]:
def set_policies(policies_name, user_segment, user_features, n_playlists):
    # Please see section 3.3 of RecSys paper for a description of policies
    POLICIES_SETTINGS = {
        'random' : RandomPolicy(n_playlists),
        'etc-seg-explore' : ExploreThenCommitSegmentPolicy(user_segment, n_playlists, min_n = 100, cascade_model = True),
        'etc-seg-exploit' : ExploreThenCommitSegmentPolicy(user_segment, n_playlists, min_n = 20, cascade_model = True),
        'epsilon-greedy-explore' : EpsilonGreedySegmentPolicy(user_segment, n_playlists, epsilon = 0.1, cascade_model = True),
        'epsilon-greedy-exploit' : EpsilonGreedySegmentPolicy(user_segment, n_playlists, epsilon = 0.01, cascade_model = True),
        'kl-ucb-seg' : KLUCBSegmentPolicy(user_segment, n_playlists, cascade_model = True),
        'ts-seg-naive' : TSSegmentPolicy(user_segment, n_playlists, alpha_zero = 1, beta_zero = 1, cascade_model = True),
        'ts-seg-pessimistic' : TSSegmentPolicy(user_segment, n_playlists, alpha_zero = 1, beta_zero = 99, cascade_model = True),
        'ts-lin-naive' : LinearTSPolicy(user_features, n_playlists, bias = 0.0, cascade_model = True),
        'ts-lin-pessimistic' : LinearTSPolicy(user_features, n_playlists, bias = -5.0, cascade_model = True),
        # Versions of epsilon-greedy-explore and ts-seg-pessimistic WITHOUT cascade model
        'epsilon-greedy-explore-no-cascade' : EpsilonGreedySegmentPolicy(user_segment, n_playlists, epsilon = 0.1, cascade_model = False),
        'ts-seg-pessimistic-no-cascade' : TSSegmentPolicy(user_segment, n_playlists, alpha_zero = 1, beta_zero = 99, cascade_model = False)
    }

    return [POLICIES_SETTINGS[name] for name in policies_name]

In [9]:
po = 'random,ts-lin-pessimistic,ts-seg-pessimistic'
policies_name = po.split(",")

In [10]:
policies = set_policies(policies_name, user_segment, user_features, n_playlists)

In [11]:
policies_name

['random', 'ts-lin-pessimistic', 'ts-seg-pessimistic']

In [12]:
n_policies = len(policies) # 3
n_users_per_round = args.n_users_per_round
n_rounds = args.n_rounds
overall_rewards = np.zeros((n_policies, n_rounds))
overall_optimal_reward = np.zeros(n_rounds)

In [13]:
cont_env = ContextualEnvironment(user_features, playlist_features, user_segment, args.n_recos)

In [14]:
cont_env.th_rewards

array([0.51069545, 0.60661954, 0.45462272, 0.35597582, 0.45974146,
       0.33826388, 0.94396647, 0.6756706 , 0.35810638])

In [15]:
user_ids = np.random.choice(range(n_users), n_users_per_round) # range * 수

In [16]:
np.take(cont_env.th_rewards, user_ids).sum()

10452.793726635924

In [17]:
n_users = 9
u = 0
step = 100000

In [18]:
probas = np.take(user_features, range(0,9), axis = 0).dot(playlist_features.T)

In [19]:
so = np.argsort(-probas)[:, :12]

In [20]:
so1 = np.take(playlist_features, so, axis = 0)

In [21]:
batch_user_ids = range(0,9)
batch_recos = so
batch_user_features = np.take(user_features, batch_user_ids, axis = 0)              # batch 안에 있는 user에 대한 user_feature만 추출
batch_playlist_features = np.take(playlist_features, batch_recos, axis = 0)    # 추천리스트에 있는 item에 대한 feature 추출
n_users = len(batch_user_ids)                                                       # batch 내에 있는 유저 수
th_reward = np.zeros(n_users)                                                       # zero vector 생성 (user 수)
for i in range(n_users):                                                            # user 수 만큼 for 문 수행
    probas = expit(batch_user_features[i].dot(batch_playlist_features[i].T)) 
    th_reward[i] = 1 - reduce(lambda x,y : x * y, 1 - probas)

In [22]:
batch_user_features[i].dot(batch_playlist_features[i].T)

array([-2.66080412, -2.93343552, -2.95593961, -3.30119431, -3.36031116,
       -3.42852429, -3.46619029, -3.47532706, -3.49337995, -3.56729809,
       -3.62311179, -3.65233738])

In [23]:
expit(batch_user_features[i].dot(batch_playlist_features[i].T)) 

array([0.06532622, 0.05052526, 0.04945654, 0.03553024, 0.03355913,
       0.03141581, 0.03028968, 0.03002246, 0.02950118, 0.02745687,
       0.02600514, 0.02527506])

In [24]:
for i in range(9):                                                            # user 수 만큼 for 문 수행
    probas = expit(np.take(user_features, range(0,9), axis = 0)[i].dot(so1[i].T))        # sigmoid 
    th_reward[i] = 1 - reduce(lambda x,y : x * y, 1 - probas)

In [25]:
th_rewards = np.zeros(user_features.shape[0]) 

In [26]:
th_rewards[0:9] = th_reward

In [27]:
user_segment = np.array(users_df.segment)

In [28]:
user_segment

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [29]:
n_segments = len(np.unique(user_segment))                              # unique 한 segment 수
segment_recos = np.zeros((n_segments, 12), dtype = np.int64)                 # (segment 수, 추천리스트 수) > segment 별로 동일한 추천리스트 생성
for i in range(n_segments):
    mean_probas = np.mean(expit(np.take(user_features, np.where(user_segment == i)[0], axis = 0).dot(playlist_features.T)), axis = 0)
    reward = 1 - reduce(lambda x,y : x * y, 1 + np.sort(-mean_probas)[:12])
    segment_recos[i] = np.argsort(-mean_probas)[:12]

In [30]:
segment_recos

array([[195, 371,  51, 408, 128, 413, 251, 718, 396, 172, 178, 252]],
      dtype=int64)

In [31]:
users_ids = range(u, min(n_users, u + step))

In [32]:
user_segment = np.take(user_segment, users_ids)                    # batch 안에 있는 user의 segment 추출
opt_recos = np.take(segment_recos, user_segment, axis = 0)

In [33]:
th_rewards

array([0.51069545, 0.60661954, 0.45462272, 0.35597582, 0.45974146,
       0.33826388, 0.94396647, 0.6756706 , 0.35810638])

In [34]:
user_ids = np.random.choice(range(n_users), n_users_per_round)                                  # 전체 유저에서 batch 크기 만큼 샘플링 / 중복 유저도 가능한데...?
overall_optimal_reward = np.take(cont_env.th_rewards, user_ids).sum()

In [35]:
for j in range(n_policies):
    # Compute n_recos recommendations
    recos = policies[j].recommend_to_users_batch(user_ids, args.n_recos, args.l_init)
    # Compute rewards
    rewards = cont_env.simulate_batch_users_reward(batch_user_ids= user_ids, batch_recos=recos)
    # Update policy based on rewards
    policies[j].update_policy(user_ids, recos, rewards, args.l_init)
    overall_rewards[j,i] = rewards.sum()
# Print info
if i == 0 or (i+1) % print_every == 0 or i+1 == n_rounds:
    logger.info("Round: %d/%d. Elapsed time: %f sec." % (i+1, n_rounds, time.time() - start_time))
    logger.info("Cumulative regrets: \n%s \n" % "\n".join(["	%s : %s" % (policies_name[j], str(np.sum(overall_optimal_reward - overall_rewards[j]))) for j in range(n_policies)]))



KeyboardInterrupt: 

In [155]:
recos = policies[2].recommend_to_users_batch(user_ids, args.n_recos, args.l_init)

In [36]:
recos.shape

(20000, 12)

In [37]:
def __init__(self, user_features, n_playlists, bias=0.0, cascade_model=True):                       # LinearTSPolicy(user_features, n_playlists, bias = -5.0, cascade_model = True)
    self.user_features = user_features
    n_dim = user_features.shape[1]                                                                  # user feature dimension : 97
    self.n_playlists = n_playlists                                                                  # item 수 : 862
    self.models = [OnlineLogisticRegression(1, 1, n_dim, bias, 15) for i in range(n_playlists)]     # item
    self.m = np.zeros((n_playlists, n_dim))                                                         # item feature matrix
    self.m[:, -1] = bias
    self.q = np.ones((n_playlists, n_dim))
    self.n_dim = n_dim
    self.cascade_model = cascade_model

In [38]:
OnlineLogisticRegression(1, 1, 97, 0, 15).w

array([-0.1158134 ,  1.29558113, -0.28211127,  0.98817131, -0.84171016,
       -1.24249283, -0.3698528 , -1.55863396,  0.0488425 ,  1.78548942,
       -1.58999274,  0.40646193,  0.59608539, -1.13041109,  0.12511812,
       -1.36892772, -0.97075994, -0.79518606, -1.12164342,  0.11741949,
        1.09571538,  1.05776025, -0.24244889, -0.55902789, -0.70053593,
       -2.79987688, -0.24961536, -0.09604944, -1.40635191,  0.54144851,
       -1.17314381, -0.59801933,  0.76765471,  0.24713131, -2.07375419,
       -2.21565849,  0.32279039, -0.79780085, -0.57758402,  0.64455904,
        0.14332806,  1.40615036, -1.12842768,  0.18442381,  1.18790698,
       -1.93552802, -0.24814258,  0.41294158, -1.34486367, -0.23577927,
       -1.26214386, -0.29943633,  0.55351547, -1.72487618,  0.26888226,
        1.41249848,  0.10417268, -1.48647481,  0.20912047, -0.80073322,
        1.13456397,  0.92475428,  1.21993949,  1.66018573, -0.1799777 ,
       -0.44706051, -0.5238736 ,  0.50986446,  0.37488435,  0.79

In [39]:
user_features = np.take(self.user_features, batch_users, axis=0)                                # batch 내 user의 feature 추출
n_users = len(batch_users)                                                                      # batch 내 유저 수
recos = np.zeros((n_users, n_recos), dtype=np.int64)                                            # 추천리스트 init 생성
step = 1
u = 0
while u < n_users:
    u_next = min(n_users, u+step)
    p_features_sampled =(np.random.normal(self.m, 1/np.sqrt(self.q), size= (u_next-u, self.n_playlists, self.n_dim)))   ###### 시작
    step_p = p_features_sampled.dot(user_features[u:u_next].T)
    for i in range(u_next - u):
        recos[u+i] = np.argsort((-step_p[i,:,i]))[:n_recos]
    u += step
# Shuffle l_init first slots
np.random.shuffle(recos[0:l_init])
return recos

NameError: name 'self' is not defined

In [40]:
m = np.zeros((782, 97))                                                         # item feature matrix
q = np.ones((782, 97))

In [41]:
p_features_sampled = np.random.normal(m,q,size=(1,782,97))

In [42]:
p_features_sampled.shape

(1, 782, 97)

In [43]:
step_p = p_features_sampled.dot(user_features[u:1].T)

In [44]:
np.argsort((-step_p[i,:,i]))[:12]

array([213,  71,  28, 304, 631, 464, 284, 660, 367, 588, 558, 222],
      dtype=int64)

In [45]:
recos = np.zeros((20000, 12), dtype=np.int64)

In [95]:
batch_user_features = np.take(user_features, user_ids, axis = 0)             # 샘플된 유저의 features  (s_user, 97)
batch_playlist_features = np.take(playlist_features, recos, axis = 0)        # 추천리스트의 item feature
n_users = len(user_ids)
n = len(batch_recos[0])
probas = np.zeros((n_users, n))
for i in range(n_users):
    probas[i] = expit(batch_user_features[i].dot(batch_playlist_features[i].T)) # probability to stream each reco
rewards = np.zeros((n_users, n))
i = 0
rewards_uncascaded = np.random.binomial(1, probas) # drawing rewards from probabilities
positive_rewards = set()

In [96]:
nz = rewards_uncascaded.nonzero()

In [97]:
positive_rewards = set()

In [98]:
rewards[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [99]:
nz       # 1681

(array([    1,     5,    19, ..., 19983, 19987, 19994], dtype=int64),
 array([1, 3, 3, ..., 5, 8, 3], dtype=int64))

In [100]:
for i in range(len(nz[0])):
    if nz[0][i] not in positive_rewards:
        rewards[nz[0][i]][nz[1][i]] = 1
        positive_rewards.add(nz[0][i])

In [142]:
rewards = 2*rewards - 1
batch_size = len(user_ids)
modified_playlists ={}
for i in range(batch_size):
    total_stream = len(rewards[i].nonzero())
    nb_display = 0
    for p, r in zip(recos[i], rewards[i]):
        nb_display +=1
        if p not in modified_playlists:
            modified_playlists[p] = {"X" : [], "Y" : []}
        modified_playlists[p]["X"].append(user_features[user_ids[i]])
        modified_playlists[p]["Y"].append(r)
        if ((total_stream == 0 and nb_display == 3) or (r == 1)):
            break
for p,v in modified_playlists.items():
    X = np.array(v["X"])
    Y = np.array(v["Y"])
    models[p].fit(X,Y)
    m[p] = models[p].m
    q[p] = models[p].q

In [154]:
recos[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
for p, r in zip(recos[i], rewards[i]):
    print(p)

In [145]:
modified_playlists[0]

{'X': [array([ 1.73900318e+00,  1.54620051e+00,  7.56945729e-01,  2.24068356e+00,
          6.36114955e-01, -2.32727075e+00, -6.84069812e-01,  1.32330310e+00,
         -3.23489404e+00,  2.57796550e+00,  2.40205973e-01, -8.86969045e-02,
          5.70372701e-01,  1.55649853e+00,  5.92273891e-01, -6.32919550e-01,
          1.33865249e+00,  2.43992165e-01, -2.53360081e+00,  1.83516765e+00,
         -1.57762074e+00, -1.54075336e+00, -8.70875791e-02,  8.75474453e-01,
         -2.99137890e-01, -1.73771787e+00,  1.54518032e+00, -5.31385839e-02,
         -8.59105706e-01,  3.49583805e-01,  1.68722212e+00, -2.64799595e-01,
          2.46780539e+00, -6.25650525e-01, -8.47333074e-01, -9.19546068e-01,
          2.62318301e+00,  2.11557078e+00,  9.97174859e-01, -2.51794672e+00,
          2.32182002e+00,  9.46822017e-03, -8.55474472e-02,  2.09209874e-01,
          2.73096472e-01,  6.02729917e-01, -4.90326285e-01, -1.39623189e+00,
          4.39023748e-02,  6.58356726e-01,  8.50678086e-01,  3.90919328

In [102]:
rewards = 2*rewards - 1

In [111]:
modified_playlists ={}
total_stream = len(rewards[i].nonzero())
nb_display = 0
for p, r in zip(recos[1], rewards[1]):
    print(p)
    print(r)
    nb_display +=1
    if p not in modified_playlists:
        modified_playlists[p] = {"X" : [], "Y" : []}
    modified_playlists[p]["X"].append(user_features[user_ids[1]])   # i번째 유저의 feature
    modified_playlists[p]["Y"].append(r)
    if ((total_stream == 0 and nb_display == l_init) or (r == 1)):
        break

0
-1.0
0
1.0


In [126]:
for p,v in modified_playlists.items():
    print(p)
    print(v)
    #X = np.array(v["X"])
    #Y = np.array(v["Y"])
    #self.models[p].fit(X,Y)
    #self.m[p] = self.models[p].m
    #self.q[p] = self.models[p].q

0


In [118]:
models = [OnlineLogisticRegression(1, 1, user_features.shape[1], -5, 15) for i in range(n_playlists)]

In [137]:
models[0].m

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -5.])

In [ ]:
models[p].fit(X,Y)